In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import seaborn as sns

In [2]:
cd /content/drive/My Drive/Kaggles Price Predictions

/content/drive/My Drive/Kaggles Price Predictions


In [3]:
df = pd.read_csv("./HousePrices.csv",low_memory=False)

In [4]:
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,4,65.0,8450,2,0,4,4,1,5,1,6,3,3,1,6,7,5,2003,2003,2,2,13,14,2,196.0,3,5,3,3,4,4,3,706,6,0,150,856,2,...,2,5,856,854,0,1710,1,0,2,1,3,1,3,8,7,0,0,2,2003.0,2,2,548,5,5,3,0,61,0,0,0,0,0,0,0,0,2,2008,9,5,208500
1,2,20,4,80.0,9600,2,0,4,4,1,3,1,25,2,3,1,3,6,8,1976,1976,2,2,9,9,3,0.0,4,5,2,3,4,2,1,978,6,0,284,1262,2,...,2,5,1262,0,0,1262,0,1,2,0,3,1,4,6,7,1,5,2,1976.0,2,2,460,5,5,3,298,0,0,0,0,0,0,0,0,0,5,2007,9,5,181500
2,3,60,4,68.0,11250,2,0,1,4,1,5,1,6,3,3,1,6,7,5,2001,2002,2,2,13,14,2,162.0,3,5,3,3,4,3,3,486,6,0,434,920,2,...,2,5,920,866,0,1786,1,0,2,1,3,1,3,6,7,1,5,2,2001.0,2,2,608,5,5,3,0,42,0,0,0,0,0,0,0,0,9,2008,9,5,223500
3,4,70,4,60.0,9550,2,0,1,4,1,1,1,7,3,3,1,6,7,5,1915,1970,2,2,14,16,3,0.0,4,5,1,4,2,4,1,216,6,0,540,756,2,...,2,5,961,756,0,1717,1,0,1,0,3,1,3,7,7,1,3,6,1998.0,3,3,642,5,5,3,0,35,272,0,0,0,0,0,0,0,2,2006,9,1,140000
4,5,60,4,84.0,14260,2,0,1,4,1,3,1,16,3,3,1,6,8,5,2000,2000,2,2,13,14,2,350.0,3,5,3,3,4,1,3,655,6,0,490,1145,2,...,2,5,1145,1053,0,2198,1,0,2,1,4,1,3,9,7,1,5,2,2000.0,2,3,836,5,5,3,192,84,0,0,0,0,0,0,0,0,12,2008,9,5,250000


In [5]:
len(df)

1460

# Ok we have imported manipulated csv

In [36]:
from sklearn.metrics import mean_squared_error
import math

def rmsle(y_test,y_preds):
    return np.sqrt(mean_squared_log_error(y_test,y_preds))

def show_scores(model):
    train_preds = model.predict(X_train)
    val_preds = model.predict(X_test)
    scores = {
              "Training RMSE": math.sqrt(mean_squared_error(y_train,train_preds)),
              "Valid RMSE": math.sqrt(mean_squared_error(y_test,val_preds)),
    }
    return scores

## The only value that we need to normilize is Lot Area i think

In [8]:
X = df.drop(['SalePrice','Id'],axis=1)
X.LotArea = X.LotArea/X.LotArea.max()
y = df.SalePrice

In [9]:
X.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,4,65.0,0.039258,2,0,4,4,1,5,1,6,3,3,1,6,7,5,2003,2003,2,2,13,14,2,196.0,3,5,3,3,4,4,3,706,6,0,150,856,2,1,2,5,856,854,0,1710,1,0,2,1,3,1,3,8,7,0,0,2,2003.0,2,2,548,5,5,3,0,61,0,0,0,0,0,0,0,0,2,2008,9,5
1,20,4,80.0,0.044600,2,0,4,4,1,3,1,25,2,3,1,3,6,8,1976,1976,2,2,9,9,3,0.0,4,5,2,3,4,2,1,978,6,0,284,1262,2,1,2,5,1262,0,0,1262,0,1,2,0,3,1,4,6,7,1,5,2,1976.0,2,2,460,5,5,3,298,0,0,0,0,0,0,0,0,0,5,2007,9,5
2,60,4,68.0,0.052266,2,0,1,4,1,5,1,6,3,3,1,6,7,5,2001,2002,2,2,13,14,2,162.0,3,5,3,3,4,3,3,486,6,0,434,920,2,1,2,5,920,866,0,1786,1,0,2,1,3,1,3,6,7,1,5,2,2001.0,2,2,608,5,5,3,0,42,0,0,0,0,0,0,0,0,9,2008,9,5
3,70,4,60.0,0.044368,2,0,1,4,1,1,1,7,3,3,1,6,7,5,1915,1970,2,2,14,16,3,0.0,4,5,1,4,2,4,1,216,6,0,540,756,2,3,2,5,961,756,0,1717,1,0,1,0,3,1,3,7,7,1,3,6,1998.0,3,3,642,5,5,3,0,35,272,0,0,0,0,0,0,0,2,2006,9,1
4,60,4,84.0,0.066250,2,0,1,4,1,3,1,16,3,3,1,6,8,5,2000,2000,2,2,13,14,2,350.0,3,5,3,3,4,1,3,655,6,0,490,1145,2,1,2,5,1145,1053,0,2198,1,0,2,1,4,1,3,9,7,1,5,2,2000.0,2,3,836,5,5,3,192,84,0,0,0,0,0,0,0,0,12,2008,9,5


In [11]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)
model = RandomForestRegressor(n_jobs=-1)
grid = {"n_estimators":[10,100,500,1000,1500],
        "max_depth":[5,10,25,50,80],
        "min_samples_split":[1,5,10,20],
        "min_samples_leaf":[1,5,10,20]}
optimizedModel = GridSearchCV(estimator=model,param_grid=grid,scoring='neg_root_mean_squared_error',cv=5,verbose=2,n_jobs=-1)

In [12]:
optimizedModel.fit(X_train,y_train)optimizedModel.fit(X_train,y_train)

Fitting 5 folds for each of 400 candidates, totalling 2000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   16.1s
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:  5.6min
[Parallel(n_jobs=-1)]: Done 361 tasks      | elapsed: 13.7min
[Parallel(n_jobs=-1)]: Done 644 tasks      | elapsed: 28.8min
[Parallel(n_jobs=-1)]: Done 1009 tasks      | elapsed: 49.3min
[Parallel(n_jobs=-1)]: Done 1454 tasks      | elapsed: 72.4min
[Parallel(n_jobs=-1)]: Done 1981 tasks      | elapsed: 99.9min
[Parallel(n_jobs=-1)]: Done 2000 out of 2000 | elapsed: 101.3min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mse', max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators=100, n_jobs=-1,
                                             oob_score=False, random_state=None,
                                             verbose=0, warm_start=False),
             iid='deprecated', n_jobs

In [13]:
optimizedModel.best_params_

{'max_depth': 50,
 'min_samples_leaf': 1,
 'min_samples_split': 5,
 'n_estimators': 500}

In [14]:
show_scores(optimizedModel)

{'Train R^2': 0.9748312818663524,
 'Training MAE': 7308.37032442905,
 'Training RMSLE': 0.0668493166061898,
 'Valid MAE': 17600.187011882976,
 'Valid RMSLE': 0.14876233727444146,
 'Valid R^2': 0.8427598074857278}

In [15]:
y_preds = optimizedModel.predict(X_test)

In [16]:
y_preds[:10]

array([313492.20157641, 335056.85151017, 129309.33958997, 171041.83509141,
       192562.4098386 , 124157.35964861, 142255.98797648, 183457.18075613,
       166194.52064531, 193575.93446909])

In [17]:
diff = abs(y_test.values[:10] - y_preds[:10])

In [18]:
diff

array([11507.79842359, 15056.85151017,   690.66041003,  2958.16490859,
       10437.5901614 ,  4792.64035139, 11244.01202352, 23542.81924387,
        6694.52064531, 12424.06553091])

In [28]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

new_df = pd.read_csv("./HousePricesBestCorr.csv",low_memory=False)

X = new_df.drop(['SalePrice'],axis=1)
y = new_df.SalePrice

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

logisticModel = LogisticRegression(max_iter=10**9,verbose=2,n_jobs=-1)
logisticModel.fit(X_train,y_train)
y_preds_logistic = logisticModel.predict(X_test)
print('RMSE is', math.sqrt(mean_squared_error(y_test,y_preds_logistic)))

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


RMSE is 56358.52323222999


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed: 10.5min finished


In [37]:
show_scores(logisticModel)

{'Training RMSE': 23910.428496981116, 'Valid RMSE': 56358.52323222999}

In [39]:
model = RandomForestRegressor(**optimizedModel.best_params_)
model.fit(X_train,y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=50, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=5, min_weight_fraction_leaf=0.0,
                      n_estimators=500, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [40]:
show_scores(model)

{'Training RMSE': 13751.60783580971, 'Valid RMSE': 36942.373559318665}

## Random Forest is better than Logistic Regression but yet not still good